# What is PyPSA?

**Python for Power System Analysis (PyPSA)**

This notebook introduces PyPSA, a free software toolbox for simulating and optimizing modern power systems that include features such as conventional generators with unit commitment, variable wind and solar generation, storage units, coupling to other energy sectors, and mixed alternating and direct current networks.

## Learning Objectives

By the end of this notebook, you will:
1. Understand what PyPSA is and its core capabilities
2. Learn the basic components of a power system model in PyPSA
3. Build your first simple power system network
4. Run a basic power flow analysis
5. Visualize network topology and results

## 1. Introduction to PyPSA

PyPSA stands for "Python for Power System Analysis". It's an open-source toolbox that enables researchers and engineers to model, simulate, and optimize electrical power systems. 

### Key Features:
- **Open Source**: Free to use and modify
- **Flexible**: Supports AC/DC networks, storage, renewable integration
- **Scalable**: From small test systems to continental-scale networks
- **Optimization**: Built-in support for linear and mixed-integer optimization
- **Time Series**: Handles multi-period optimization problems
- **Visualization**: Built-in plotting capabilities

In [ ]:
# Import necessary libraries
import pypsa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Check PyPSA version
print(f"PyPSA version: {pypsa.__version__}")

## 2. Core Components of PyPSA

A PyPSA power system model consists of several key components:

1. **Network**: The main container for all components
2. **Buses**: Electrical nodes where components connect
3. **Generators**: Power production units (conventional, renewable)
4. **Loads**: Power consumption
5. **Lines**: Transmission lines connecting buses
6. **Transformers**: Voltage level connections
7. **Storage**: Energy storage units
8. **Links**: General connections (DC lines, sector coupling)

## 3. Building Your First Network

Let's create a simple 3-bus power system to demonstrate PyPSA's capabilities.

In [ ]:
# Create a new network
network = pypsa.Network(name="Simple 3-Bus System")

# Add buses
network.add("Bus", "Bus1", v_nom=230, x=0, y=1)  # Generator bus
network.add("Bus", "Bus2", v_nom=230, x=1, y=0)  # Load bus 1  
network.add("Bus", "Bus3", v_nom=230, x=2, y=1)  # Load bus 2

print("Buses added:")
print(network.buses)

In [ ]:
# Add generators
network.add("Generator", "Coal", 
           bus="Bus1", 
           p_nom=1000,  # Nominal power in MW
           marginal_cost=50,  # $/MWh
           carrier="coal")

network.add("Generator", "Gas", 
           bus="Bus1", 
           p_nom=800, 
           marginal_cost=80,
           carrier="gas")

# Add a renewable generator (wind)
network.add("Generator", "Wind", 
           bus="Bus2", 
           p_nom=400, 
           marginal_cost=0,
           carrier="wind")

print("\nGenerators added:")
print(network.generators)

In [ ]:
# Add loads
network.add("Load", "Load1", 
           bus="Bus2", 
           p_set=600)  # Load demand in MW

network.add("Load", "Load2", 
           bus="Bus3", 
           p_set=400)

print("Loads added:")
print(network.loads)

In [ ]:
# Add transmission lines
network.add("Line", "Line1-2", 
           bus0="Bus1", 
           bus1="Bus2", 
           x=0.1,  # Reactance
           r=0.01,  # Resistance
           s_nom=1200)  # Power rating in MVA

network.add("Line", "Line1-3", 
           bus0="Bus1", 
           bus1="Bus3", 
           x=0.15, 
           r=0.02, 
           s_nom=800)

network.add("Line", "Line2-3", 
           bus0="Bus2", 
           bus1="Bus3", 
           x=0.12, 
           r=0.015, 
           s_nom=600)

print("Lines added:")
print(network.lines)

## 4. Network Visualization

Let's visualize our network topology:

In [ ]:
# Plot the network
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the network topology
network.plot(ax=ax, 
            bus_sizes=1000,
            line_widths=3,
            title="Simple 3-Bus Power System")

plt.show()

## 5. Running Power Flow Analysis

Now let's solve the power flow to determine how power flows through our network:

In [ ]:
# Run linear power flow (LOPF - Linear Optimal Power Flow)
network.lopf()

print("Network optimization completed!")
print(f"Objective value: ${network.objective:.2f}")

In [ ]:
# Display generator dispatch results
print("Generator Dispatch:")
print("====================")
for gen_name in network.generators.index:
    power = network.generators_t.p.loc[network.snapshots[0], gen_name]
    capacity = network.generators.loc[gen_name, 'p_nom']
    cost = network.generators.loc[gen_name, 'marginal_cost']
    print(f"{gen_name}: {power:.1f} MW ({power/capacity*100:.1f}% of capacity) at ${cost}/MWh")

In [ ]:
# Display line flows
print("\nLine Flows:")
print("============")
for line_name in network.lines.index:
    flow = network.lines_t.p0.loc[network.snapshots[0], line_name]
    capacity = network.lines.loc[line_name, 's_nom']
    utilization = abs(flow) / capacity * 100
    print(f"{line_name}: {flow:.1f} MW ({utilization:.1f}% utilization)")

## 6. Visualizing Results

Let's create some visualizations to better understand our results:

In [ ]:
# Create a bar chart of generator dispatch
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Generator dispatch
gen_power = [network.generators_t.p.loc[network.snapshots[0], gen] 
            for gen in network.generators.index]
gen_colors = ['brown', 'gray', 'green']

bars = ax1.bar(network.generators.index, gen_power, color=gen_colors)
ax1.set_ylabel('Power Output (MW)')
ax1.set_title('Generator Dispatch')
ax1.grid(True, alpha=0.3)

# Add values on bars
for bar, power in zip(bars, gen_power):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
             f'{power:.0f} MW', ha='center', va='bottom')

# Line utilization
line_flows = [abs(network.lines_t.p0.loc[network.snapshots[0], line]) 
             for line in network.lines.index]
line_capacities = network.lines['s_nom'].values
utilizations = [flow/cap*100 for flow, cap in zip(line_flows, line_capacities)]

bars2 = ax2.bar(network.lines.index, utilizations, color='blue', alpha=0.7)
ax2.set_ylabel('Line Utilization (%)')
ax2.set_title('Transmission Line Utilization')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 100)

# Add values on bars
for bar, util in zip(bars2, utilizations):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{util:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 7. Adding Time Series Analysis

Let's extend our analysis to include time-varying loads and renewable generation:

In [ ]:
# Create a new network with time series
network_ts = pypsa.Network(name="3-Bus System with Time Series")

# Set up time snapshots (24 hours)
snapshots = pd.date_range('2023-01-01', periods=24, freq='H')
network_ts.set_snapshots(snapshots)

# Add buses (same as before)
network_ts.add("Bus", "Bus1", v_nom=230, x=0, y=1)
network_ts.add("Bus", "Bus2", v_nom=230, x=1, y=0) 
network_ts.add("Bus", "Bus3", v_nom=230, x=2, y=1)

# Add generators
network_ts.add("Generator", "Coal", bus="Bus1", p_nom=1000, marginal_cost=50)
network_ts.add("Generator", "Gas", bus="Bus1", p_nom=800, marginal_cost=80)

# Add wind with time-varying availability
wind_profile = 0.3 + 0.4 * np.sin(np.linspace(0, 2*np.pi, 24)) + 0.1 * np.random.randn(24)
wind_profile = np.clip(wind_profile, 0, 1)  # Ensure between 0 and 1

network_ts.add("Generator", "Wind", bus="Bus2", p_nom=400, marginal_cost=0,
              p_max_pu=wind_profile)  # Time-varying capacity factor

# Add lines
network_ts.add("Line", "Line1-2", bus0="Bus1", bus1="Bus2", x=0.1, r=0.01, s_nom=1200)
network_ts.add("Line", "Line1-3", bus0="Bus1", bus1="Bus3", x=0.15, r=0.02, s_nom=800)
network_ts.add("Line", "Line2-3", bus0="Bus2", bus1="Bus3", x=0.12, r=0.015, s_nom=600)

# Add loads with daily demand profile
load_profile = 0.6 + 0.3 * np.sin(np.linspace(0, 2*np.pi, 24) - np.pi/2)  # Peak at noon
network_ts.add("Load", "Load1", bus="Bus2", p_set=600 * load_profile)
network_ts.add("Load", "Load2", bus="Bus3", p_set=400 * load_profile)

print("Time series network created with 24 hourly snapshots")

In [ ]:
# Visualize the load and wind profiles
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Load profile
hours = range(24)
ax1.plot(hours, 600 * load_profile, 'b-', label='Load 1', linewidth=2)
ax1.plot(hours, 400 * load_profile, 'r-', label='Load 2', linewidth=2)
ax1.set_ylabel('Load Demand (MW)')
ax1.set_title('Daily Load Profile')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Wind profile
ax2.plot(hours, wind_profile * 400, 'g-', label='Wind Available', linewidth=2)
ax2.axhline(y=400, color='g', linestyle='--', alpha=0.5, label='Max Wind Capacity')
ax2.set_xlabel('Hour of Day')
ax2.set_ylabel('Wind Power (MW)')
ax2.set_title('Wind Generation Profile')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Run the time series optimization
network_ts.lopf()

print(f"Time series optimization completed!")
print(f"Total system cost: ${network_ts.objective:.2f}")

In [ ]:
# Plot the generation dispatch over time
fig, ax = plt.subplots(figsize=(14, 8))

# Get generation data
gen_data = network_ts.generators_t.p
load_data = network_ts.loads_t.p.sum(axis=1)

# Create stacked area plot
ax.fill_between(hours, 0, gen_data['Coal'], label='Coal', color='brown', alpha=0.8)
ax.fill_between(hours, gen_data['Coal'], 
               gen_data['Coal'] + gen_data['Gas'], 
               label='Gas', color='gray', alpha=0.8)
ax.fill_between(hours, gen_data['Coal'] + gen_data['Gas'], 
               gen_data.sum(axis=1), 
               label='Wind', color='green', alpha=0.8)

# Add load line
ax.plot(hours, load_data, 'k--', linewidth=2, label='Total Load')

ax.set_xlabel('Hour of Day')
ax.set_ylabel('Power (MW)')
ax.set_title('24-Hour Generation Dispatch vs Load')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Summary and Next Steps

### What We've Learned:

1. **PyPSA Basics**: PyPSA is a powerful tool for power system modeling and optimization
2. **Network Components**: Buses, generators, loads, and lines form the backbone of any model
3. **Optimization**: PyPSA can solve economic dispatch problems automatically
4. **Time Series**: Real-world variability in loads and renewables can be modeled
5. **Visualization**: Results can be easily plotted and analyzed

### Key Insights from Our Example:

- The optimization chooses the cheapest generation first (wind at $0/MWh)
- Coal is preferred over gas due to lower marginal cost ($50 vs $80/MWh)
- Transmission constraints can affect the optimal dispatch
- Time-varying loads and renewables create a dynamic optimization problem

### Next Steps:

1. **Learn about storage**: Add batteries to handle renewable variability
2. **Explore larger networks**: Use real-world network data
3. **Advanced constraints**: Unit commitment, ramping limits, reserves
4. **Sector coupling**: Model connections to heating, transport, and hydrogen
5. **Uncertainty**: Stochastic optimization for uncertain renewables

### Exercises for Further Learning:

1. Modify the generator costs and observe how dispatch changes
2. Add a battery storage unit to the network
3. Increase the wind capacity and see how it affects system costs
4. Create a larger network with more buses and transmission constraints

## Resources for Further Learning

- **PyPSA Documentation**: [https://pypsa.readthedocs.io/](https://pypsa.readthedocs.io/)
- **PyPSA Examples**: [https://github.com/PyPSA/pypsa-examples](https://github.com/PyPSA/pypsa-examples)
- **PyPSA-Eur**: Continental-scale model of Europe
- **AtLite**: Tool for renewable energy potential analysis
- **Research Papers**: Many studies use PyPSA for energy system analysis